# Experiments for the paper

## Preamble

In [ ]:
import jax, jax.numpy as jnp
import argparse
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime
import math
jax.config.update('jax_platform_name', 'cpu')
jax.config.update('jax_enable_x64', True)

from src.stats.hmm import get_generative_model
from src.variational import get_variational_model
from src.training import SVITrainer


In [ ]:
def set_defaults(args):
    args.default_prior_mean = 0.0 # default value for the mean of Gaussian prior
    args.default_prior_base_scale = math.sqrt(0.1) # default value for the diagonal components of the covariance matrix of the prior
    args.default_transition_base_scale = math.sqrt(0.1) # default value for the diagonal components of the covariance matrix of the transition kernel
    args.default_transition_bias = 0.0
    args.default_emission_base_scale = math.sqrt(0.1)
    args.parametrization = 'cov_chol'
    return args


def plot_x_true_against_x_pred(x_true, x_pred):
    dims = x_true.shape[-1]
    _ , axes = plt.subplots(dims, 1, figsize=(15,2*dims))
    for dim in range(dims):
        axes[dim].plot(x_true[:,dim], c='red', label='True')
        axes[dim].plot(x_pred[:,dim], c='green', label='Pred')
        axes[dim].legend()

def compute_rmse_x_true_against_x_pred(x_true, x_pred):
    rmse = jax.vmap(lambda x,y: jnp.mean(jnp.sqrt(jnp.mean((x-y)**2, axis=-1)), axis=0))
    print('RMSE:',rmse.tolist())


## Linear-Gaussian HMM 

In [ ]:
date = datetime.now().strftime('%Y_%m_%d__%H_%M_%S')
# exp_path = os.path.join('experiments', date)
# os.makedirs(exp_path, exist_ok=True)

### Setting up data and generative model

In [ ]:
key = jax.random.PRNGKey(0) # seed
p_args = argparse.Namespace() # will contain all configuration information for the generative model 
p_args.state_dim, p_args.obs_dim = 10,10
p_args.model = 'linear' 
p_args.seq_length = 500
p_args.emission_bias = False # no bias in the emission kernel, i.e. only Y_t = B X_t + noise
p_args.transition_bias = False # no bias in the transition kernel, i.e. only X_t = A X_{t-1} + noise
p_args.transition_matrix_conditionning = 'diagonal' # the transition matrix will be diagonal
p_args.range_transition_map_params = [0.8,0.9] # the diagonal values are constrained to [0.9 1]
p_args.num_seqs = 1 #
p_args = set_defaults(p_args) # setting default values for the covariances with realistic Signal to noise ratios

key, key_theta, key_sequences = jax.random.split(key, 3)
p, theta = get_generative_model(p_args, key_theta) # getting the model and its parameters 
print(theta)
# save_args(p_args, 'args', exp_path) # saving the params of the generative model
# save_params(theta, 'theta', exp_path) # saving the parameters
# jnp.save(os.path.join(exp_path, 'xs'), xs)
# jnp.save(os.path.join(exp_path, 'ys'), ys)

In [ ]:
xs, ys = p.sample_multiple_sequences(key_sequences, 
                                     theta, 
                                     p_args.num_seqs, 
                                     p_args.seq_length,
                                     single_split_seq=False, # not sampling a sequence and splitting into subsequences
                                     load_from='', # not loading from external folder
                                     loaded_seq=False) # not loading sequences

### Setting up variational model and training parameters

In [ ]:
q_args = argparse.Namespace() # will contain all configuration information for the variational model 
q_args.state_dim, q_args.obs_dim = p_args.state_dim, p_args.obs_dim 
q_args.model = 'linear' 
q_args.emission_bias = False 
q_args.transition_bias = False 
q_args.transition_matrix_conditionning = 'diagonal' 
q_args.range_transition_map_params = [0.8,0.9] 
q_args = set_defaults(q_args) 
# sub_exp_path = os.path.join(exp_path, q_args.model)
# os.makedirs(sub_exp_path, exist_ok=True)
# save_args(p_args, 'args', sub_exp_path) # saving the params of the generative model
q = get_variational_model(q_args) # getting the corresponding model


### Learning in an offline setting

In [ ]:
key = jax.random.PRNGKey(1)
for elbo_mode in ['closed_form', 'autodiff_on_batch', 'score,paris,bptt_depth_2']:
    trainer = SVITrainer(p=p,
                        theta_star=theta,
                        q=q,
                        optimizer='adam',
                        learning_rate=1e-3, 
                        optim_options='cst', # learning rate schedule
                        num_epochs=1000, # number of full sweeps through the sequence
                        seq_length=p_args.seq_length,
                        num_samples=2, # number of monte carlo samples (or trajectories in the offline case)
                        frozen_params='', # which parameters to hold fixed
                        num_seqs=p_args.num_seqs, 
                        training_mode=f'reset,{p_args.seq_length},1', 
                        elbo_mode=elbo_mode,
                        logging_type='basic_logging')


    key_init_params, key_montecarlo = jax.random.split(key, 2)
    fitted_variational_params, elbos = trainer.fit(key_init_params, 
                                    key_montecarlo, 
                                    data=(xs, ys), 
                                    log_writer=None, 
                                    args=None, 
                                    log_writer_monitor=None)

    elbo_for_all_epochs = elbos.flatten()
    plt.plot(elbo_for_all_epochs, label=elbo_mode)
plt.legend()

In [ ]:
print(fitted_variational_params)
x_smoothed_kalman = p.smooth_seq(ys[0], theta)[0]
x_smoothed_variational = q.smooth_seq(ys[0], fitted_variational_params)[0]
plot_x_true_against_x_pred(x_smoothed_kalman, x_smoothed_variational)

### Streaming data

In [ ]:
p_args.seq_length = 100_000 # keeping all previous settings except the sequence length

xs, ys = p.sample_multiple_sequences(key_sequences, 
                                     theta, 
                                     num_seqs=1, 
                                     seq_length=p_args.seq_length,
                                     single_split_seq=False,
                                     load_from='')


elbo_mode = 'score,paris,bptt_depth_2'
trainer = SVITrainer(p=p,
                    theta_star=theta,
                    q=q,
                    optimizer='adam',
                    learning_rate=1e-6, 
                    optim_options='cst', # learning rate schedule
                    num_epochs=1, # number of full sweeps through the sequence
                    seq_length=p_args.seq_length,
                    num_samples=10, # number of monte carlo samples (or trajectories in the online case)
                    frozen_params='', # which parameters to hold fixed
                    num_seqs=p_args.num_seqs, 
                    training_mode=f'streaming,1,difference', # L_t - L_{t-1} / t
                    elbo_mode=elbo_mode,
                    logging_type='basic_logging')

key_init_params, key_montecarlo = jax.random.split(key, 2)
fitted_variational_params, elbos = trainer.fit(key_init_params, 
                                key_montecarlo, 
                                data=(xs, ys), 
                                log_writer=None, 
                                args=None, 
                                log_writer_monitor=None)

In [ ]:
elbo_for_all_epochs = elbos.flatten()
plt.plot(elbo_for_all_epochs, label=elbo_mode)
plt.legend()

x_smoothed_kalman = p.smooth_seq(ys[0], theta)[0]
x_smoothed_variational = q.smooth_seq(ys[0], fitted_variational_params)[0]
plot_x_true_against_x_pred(x_smoothed_kalman, x_smoothed_variational)

## Chaotic RNN

### Streaming data

In [ ]:
p_args = argparse.Namespace()

p_args.state_dim, p_args.obs_dim = 5,5 
p_args.model = 'chaotic_rnn' 
p_args.seq_length = 500_000
p_args.loaded_seq = True
p_args.load_from = 'data/crnn/2022-10-18_15-28-00_Train_run'
p_args.range_transition_map_params = [-1,1] # range of the components of the transition matrix
p_args.transition_matrix_conditionning = 'init_scale_by_dim' # constraint
p_args.default_transition_matrix = os.path.join(p_args.load_from, 'W.npy')

p_args.gamma = 2.5 # gamma for the chaotic rnn
p_args.tau = 0.025 # tau for the chaotic rnn
p_args.grid_size = 0.001 # discretization parameter for the chaotic rnn

p_args.emission_matrix_conditionning = 'diagonal'
p_args.range_emission_map_params = (-1,1)
p_args.default_emission_df = 2 # degrees of freedom for the emission noise
p_args.default_emission_matrix = 1.0 # diagonal values for the emission matrix
p_args.transition_bias = False 
p_args.emission_bias = False
p_args.num_seqs = 1 #

p_args.default_prior_mean = 0.0 # default value for the mean of Gaussian prior
p_args.default_prior_base_scale = math.sqrt(0.1) # default value for the diagonal components of the covariance matrix of the prior
p_args.default_transition_base_scale = math.sqrt(0.1) # default value for the diagonal components of the covariance matrix of the transition kernel
p_args.default_transition_bias = 0.0
p_args.default_emission_base_scale = math.sqrt(0.1)
p_args.num_particles, p_args.num_smooth_particles = None, None


In [ ]:
key = jax.random.PRNGKey(0)
key, key_params, key_sequences = jax.random.split(key, 3)
p, theta = get_generative_model(p_args, key_params)
xs, ys = p.sample_multiple_sequences(key_sequences, 
                                     theta, 
                                     1, 
                                     p_args.seq_length,
                                     single_split_seq=False)

In [ ]:
q_args = argparse.Namespace()
q_args.model = 'johnson_backward,100'
q_args.state_dim, q_args.obs_dim = p_args.state_dim, p_args.obs_dim
q_args.transition_bias = False
q_args.emission_bias = False
layers = [int(nb) for nb in q_args.model.split(',')[-1].split('_')]
q_args.update_layers = (*layers,)
q_args.backwd_layers = 0
q_args.transition_matrix_conditionning = 'diagonal'
q_args.range_transition_map_params = (0.8, 1)
q_args.anisotropic = False

q_args = set_defaults(q_args)

q = get_variational_model(q_args)



In [ ]:
key = jax.random.PRNGKey(1)
for elbo_mode in ['score,paris,bptt_depth_2']:
    trainer = SVITrainer(p=p,
                        theta_star=theta,
                        q=q,
                        optimizer='adam',
                        learning_rate=1e-3, 
                        optim_options='cst', # learning rate schedule
                        num_epochs=1, # number of full sweeps through the sequence
                        seq_length=p_args.seq_length,
                        num_samples=2, # number of monte carlo samples (or trajectories in the offline case)
                        frozen_params='', # which parameters to hold fixed
                        num_seqs=p_args.num_seqs, 
                        training_mode='accumulate,1000,1', 
                        elbo_mode=elbo_mode,
                        logging_type='basic_logging')


    key_init_params, key_montecarlo = jax.random.split(key, 2)
    fitted_variational_params, elbos = trainer.fit(key_init_params, 
                                    key_montecarlo, 
                                    data=(xs, ys), 
                                    log_writer=None, 
                                    args=None, 
                                    log_writer_monitor=None)

    elbo_for_all_epochs = elbos.flatten()
    plt.plot(elbo_for_all_epochs, label=elbo_mode)
plt.legend()

In [ ]:
x_smoothed = q.smooth_seq(ys[0], fitted_variational_params)[0]
plot_x_true_against_x_pred(x_true=xs[0], 
                           x_pred=x_smoothed)